In [1]:
!pip install -q transformers rich torchmetrics sklearn pytorch-lightning

In [2]:
import pandas as pd
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import dask.bag as db
import json
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
import csv
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torchmetrics import F1
from torchmetrics.functional import accuracy

In [3]:
!mkdir dataset

In [4]:
raw_json_data = "arxiv-metadata-oai-snapshot-json.zip"
pickled_dataset = 'dataset_pickle.pkl'
pickled_dataset_onehot = 'dataset_multilabel.pkl'

In [5]:
def __iter__(self): return 0

ibm_api_key_id = 'SECRET'
bucket = 'SECRET'

client_c2b4ce19d76d4b7c87bdf6c8c84d662c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=ibm_api_key_id,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_c2b4ce19d76d4b7c87bdf6c8c84d662c.get_object(Bucket=bucket, Key='arxiv-metadata-oai-snapshot.json.zip')['Body']

def save_to_cloud(key, file):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.upload_file(Bucket=bucket, Key=f'snapshot-{file}', Filename=file)

    
def get_file(key, filename):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.download_file(Bucket=bucket, Key=key, Filename=filename)

In [7]:
# try:
#     get_file('arxiv-metadata-oai-snapshot.pkl', 'dataset_pickle.pkl')
# except:
#     print("Exception while pulling, will recreate from scratch")
    

# if not os.path.exists(pickled_dataset):
#     get_file('arxiv-metadata-oai-snapshot.json.zip', raw_json_data)
#     with zipfile.ZipFile(raw_json_data, 'r') as zip_ref:
#         zip_ref.extractall("dataset")
#     docs = db.read_text('./dataset/arxiv-metadata-oai-snapshot.json').map(json.loads)
#     trim = lambda x: {'id': x['id'],
#                   'category':x['categories'].split(' '),
#                   'abstract':x['abstract'].replace('\t', ' ')}
#     docs_df = (docs.map(trim).compute())
#     df = pd.DataFrame(docs_df)
#     df.to_pickle(pickled_dataset)
#     save_to_cloud('arxiv-metadata-oai-snapshot.pkl', pickled_dataset)    

In [8]:
# df = pd.read_pickle('dataset_pickle.pkl')

In [9]:
# df

In [10]:
# try:
#     get_file('snapshot-dataset_multilabel.pkl', 'dataset_multilabel.pkl')
#     print("Got S3 file")
# except:
#     print("Exception while pulling, will recreate from scratch")
    
# if not os.path.exists('dataset_multilabel.pkl'):
#     mlb = MultiLabelBinarizer()
#     labels = mlb.fit_transform(df.category)
#     df_onehot = pd.concat([df[['abstract']], pd.DataFrame(labels)], axis=1)
#     df_onehot.columns = ['abstract'] + list(mlb.classes_)
#     df_onehot.to_pickle('dataset_multilabel.pkl')
#     save_to_cloud('snapshot-dataset_multilabel.pkl', 'dataset_multilabel.pkl')

In [11]:
# df_onehot = pd.read_pickle('dataset_multilabel.pkl')

In [12]:
# LABEL_COLUMNS = df_onehot.columns.tolist()[1:]
# LABEL_COLUMNS

## Trying preprocessing?

In [13]:
# get_file('arxiv-metadata-oai-snapshot.json.zip', raw_json_data)
# with zipfile.ZipFile(raw_json_data, 'r') as zip_ref:
#     zip_ref.extractall("dataset")
# docs = db.read_text('./dataset/arxiv-metadata-oai-snapshot.json').map(json.loads)


In [14]:
# get_latest_version = lambda x: x['versions'][-1]['created']
# trim = lambda x: {'id': x['id'],
#               'category':x['categories'].split(' '),
#               'abstract':x['abstract'].replace('\t', ' ')}
# docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2009).map(trim).compute())
# df = pd.DataFrame(docs_df)
# df.to_pickle("df2009.pkl")
# save_to_cloud('df2009.pkl', 'df2009.pkl')

In [15]:
# get_file('snapshot-df2009.pkl', 'df2009.pkl')
# df = pd.read_pickle('df2009.pkl')

In [16]:
# %%time
# df["abstract"] = df["abstract"].apply(lambda text: " ".join([word for word in text.replace("\n", " ").replace("\t", " ").strip().split(" ") if len(word) > 0]))

In [17]:
# frequent_categories = pd.Series(np.concatenate(df['category'])).value_counts().reset_index(name="count").query("count > 1000")["index"].tolist()

In [18]:
# len(frequent_categories)

In [19]:
# df = df[df["category"].apply(lambda cats: all(elem in frequent_categories for elem in cats))]

In [20]:
# df = df[df["abstract"].apply(lambda text: len(text.split()) > 128)]

In [21]:
# df = df.reset_index()

In [22]:
# categories_filtered = pd.Series(np.concatenate(df['category'])).value_counts()

In [23]:
# from rich.progress import track

# # Select 5k randomly, then at least 1k from each category, then drop duplicates
# uni_df = df.sample(n = 5000)
# for category, _ in track(categories_filtered.items(), total=len(categories_filtered), description="Pulling 1k from each cat"):
#     subset_df = df[df["category"].apply(lambda cats: category in cats)]
#     uni_df = pd.concat([subset_df.sample(n = min(1000, len(subset_df))), uni_df], ignore_index=True, sort=False)

In [24]:
# uni_df = uni_df.iloc[uni_df.astype(str).drop_duplicates().index]


In [25]:
# len(uni_df)

In [26]:
# uni_df = uni_df.reset_index()

In [27]:
# mlb = MultiLabelBinarizer()
# labels = mlb.fit_transform(uni_df.category)
# df_onehot = pd.concat([uni_df[['abstract']], pd.DataFrame(labels)], axis=1)
# df_onehot.columns = ['abstract'] + list(mlb.classes_)


In [28]:
# df_onehot.to_pickle('uni_df.pkl')
# save_to_cloud('uni_df.pkl', 'uni_df.pkl')

## Model

In [12]:
get_file('snapshot-cleaned_data_for_training.pkl', 'cleaned_data_for_training.pkl')

In [13]:
df_onehot = pd.read_pickle('cleaned_data_for_training.pkl')
LABEL_COLUMNS = df_onehot.columns.tolist()[1:]

In [14]:
df_onehot_subs = df_onehot

In [15]:
df_onehot_subs

,abstract,cs.LG,quant-ph,hep-ph,hep-th,cs.CV,gr-qc,cond-mat.mes-hall,cond-mat.mtrl-sci,astro-ph.CO,...,cond-mat,alg-geom,q-alg,q-bio,dg-ga,adap-org,chao-dyn,funct-an,solv-int,patt-sol
0,The defining property of an artificial physica...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,In this paper the certain 4-dimensional algebr...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,An alternative mathematics based on qualitativ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"In any cubic polynomial, the average of the sl...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"For the first time, a general fractional calcu...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416676,Precision measurements of high energy top quar...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
416677,Wide-field (> 100 deg$^2$) hard X-ray coded-ap...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
416678,Automatic evaluation of various text quality c...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
416679,Ferrimagnetic TbFe or TbFeCo amorphous alloy t...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df_onehot_subs['abstract'][200000]

'Nowadays, application of Service Oriented Architecture is increasing rapidly; especially since introduction of distributed electronic services on the web. SOA software has a modular manner and works as a collaboration of independent software components. As a result, e-service approach is sufficient for software with independent components, each of which may be developed by a different company. Such software components and their cooperation form a composite service. Agile methodologies are the best candidate for developing small software components. Composite services and its building blocks are small pieces of software, making agile methodology a perfect fit for their development. In this paper, we introduce an agile method for service composition, inspired by agile patterns and practices. Therefore, across the agile manifesto, we can develop low cost, high quality composite services quickly using this method.'

In [10]:
class ArxivAbstracts(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = df.abstract
        self.labels = df[df.columns.difference(['abstract'])] # <----------------- THIS SORTS LABELS!!!!!!!!!!!
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        encoding = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=False,
            return_attention_mask= True,
            truncation=True,
            return_tensors='pt'
          )
               
        return {
          'ids': encoding["input_ids"].flatten(),
          'mask': encoding["attention_mask"].flatten(),
          'targets': torch.tensor(self.labels.iloc[item_idx].tolist(), dtype=torch.float)  
        }

In [11]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot_subs, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

In [80]:
import pytorch_lightning as pl
class ArxivAbstractsModule(pl.LightningDataModule):
    
    def __init__(self, df_train, df_test, df_validation, tokenizer, batch_size=16, max_token_len=256):
        super().__init__()
        self.df_train = df_train
        self.df_test = df_test
        self.df_validation = df_validation
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = ArxivAbstracts(df=self.df_train, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.val_dataset = ArxivAbstracts(df=self.df_validation, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.test_dataset = ArxivAbstracts(df=self.df_test, tokenizer=self.tokenizer, max_len= self.max_token_len)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [81]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
arxiv_data = ArxivAbstractsModule(df_train, df_test, df_validation, tokenizer, max_token_len=384)
arxiv_data.setup()

In [8]:
import pytorch_lightning as pl
class ArxivAbstractsClassifier(pl.LightningModule):
    def __init__(self, n_classes, lr=2e-5):
        super().__init__()

        self.bert = BertModel.from_pretrained("bert-base-cased", return_dict=True)
        self.classifier = torch.nn.Linear(self.bert.config.hidden_size, n_classes) 
        self.lr = lr
        self.criterion = torch.nn.BCEWithLogitsLoss()
        
        self.f1_metrics = torch.nn.ModuleDict({
            'f1_weighted': F1(num_classes=len(LABEL_COLUMNS), average='weighted', threshold=0.5),       
            'f1_samples': F1(num_classes=len(LABEL_COLUMNS), average='samples', threshold=0.5), 
            'f1_micro': F1(num_classes=len(LABEL_COLUMNS), average='micro', threshold=0.5),
            'f1_macro': F1(num_classes=len(LABEL_COLUMNS), average='macro', threshold=0.5),
            'f1_weighted_t02': F1(num_classes=len(LABEL_COLUMNS), average='weighted', threshold=0.2),       
            'f1_samples_t02': F1(num_classes=len(LABEL_COLUMNS), average='samples', threshold=0.2), 
            'f1_micro_t02': F1(num_classes=len(LABEL_COLUMNS), average='micro', threshold=0.2),
            'f1_macro_t02': F1(num_classes=len(LABEL_COLUMNS), average='macro', threshold=0.2),
        })
        self.save_hyperparameters()

    def compute_metrics(self, outputs, labels):
        metrics = {}
        for name, metric in self.f1_metrics.items():
            metrics[name] = metric(outputs, labels)
        return metrics  
    
    def forward(self, ids, mask):
        output = self.bert(input_ids=ids, attention_mask=mask)
        output = self.classifier(output.pooler_output)   
        return output
    
    def loss(self, batch, prediction):
        return self.criterion(prediction, batch["targets"])
    
    def __generic_step(self, batch, batch_idx, namespace: str):
        input_ids = batch["ids"]
        attention_mask = batch["mask"]
        labels = batch["targets"]
        
        outputs = self(input_ids, attention_mask)
        predictions = torch.sigmoid(outputs)
        loss = self.loss(batch, outputs)
        metrics = self.compute_metrics(predictions, labels.int())
        
        if namespace != "pred":
            self.log_dict({f'{namespace}_loss': loss, **
                           {f'{namespace}_{k}': v for k, v in metrics.items()}})
        
        return {"loss": loss, "predictions": predictions, "labels": labels}
        
    def predict_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="pred")["predictions"]
    
    def training_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="train")["loss"]
        return loss

    def validation_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="val")["loss"]
    
    def test_step(self, batch, batch_idx):
        return self.__generic_step(batch, batch_idx, namespace="test")["loss"]

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [9]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    monitor='val_loss',
    filename='ArxivAbstractsClassifierPreprocess-{epoch:02d}-{val_loss:.2f}',
    save_top_k=2,
    verbose=True,
    mode='min'
)
logger = TensorBoardLogger("lightning_logs", name="arxiv-abstracts-logs")
# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [41]:
%%time 
N_EPOCHS = 4
model = ArxivAbstractsClassifier(n_classes=len(LABEL_COLUMNS))
trainer = Trainer(max_epochs = N_EPOCHS , 
                  gpus = 1, 
                  logger=logger, 
                  enable_checkpointing=True,
#                   callbacks=[early_stopping_callback], 
                  callbacks=[checkpoint_callback, RichProgressBar()])


trainer.fit(model, datamodule=arxiv_data)

Epoch 0    ━━━━━━━━━━━━━━━━━━━━━━━ 33/23439 0:01:08 • 13:10:23 0.49it/s loss: 0.511 v_num: 1 

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer
.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

CPU times: user 1min 6s, sys: 18.2 s, total: 1min 24s
Wall time: 1min 22s


In [10]:
model_name = "abrract_class_bert_cleaned_v1"
def save_dir_to_cloud(path, model_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                filepath = os.path.join(root,file)
                save_to_cloud(f"outs/{model_name}/{file}", filepath)

In [26]:
trainer.save_checkpoint(f"{model_name}.ckpt")
save_to_cloud(f'{model_name}.ckpt', f"{model_name}.ckpt")

In [ ]:
save_dir_to_cloud("lightning_logs", model_name)
save_dir_to_cloud("checkpoints", model_name)

In [17]:
get_file(f'snapshot-{model_name}.ckpt', f"snapshot-{model_name}.ckpt")
# trainer = Trainer()
# trainer.fit(model, datamodule=arxiv_data, ckpt_path=f"snapshot-{model_name}.ckpt")

In [18]:

model = ArxivAbstractsClassifier(n_classes=len(LABEL_COLUMNS)).load_from_checkpoint(f"snapshot-{model_name}.ckpt", n_classes=len(LABEL_COLUMNS))


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_re

In [ ]:
trainer = Trainer(gpus = 1, 
                  logger=logger,
                  callbacks=[RichProgressBar()])

In [19]:
# trainer.test(model, datamodule=arxiv_data)

In [20]:
preds = trainer.predict(model, dataloaders=arxiv_data.test_dataloader())

Predicting ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2605/2605 1:27:17 • 0:00:00 0.50it/s

In [21]:
preds = np.concatenate(preds, axis=0)

In [22]:
# Probably needs a more sophisticated thingy with model.eval + freezing
# Or just incorporating that to the model above so that loading from a checkpoint will work correctly
def predict(tresh = 0.5):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds]
    y_true = [tensor["targets"] for tensor in arxiv_data.test_dataloader()]
    y_true = np.concatenate([tensor.tolist() for tensor in y_true])
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [24]:
predict(tresh = 0.25)

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.64      0.82      0.72      1018
           4       0.79      0.84      0.82       806
           5       0.59      0.86      0.70       883
           6       0.65      0.84      0.73       944
           7       0.67      0.70      0.68       815
           8       0.70      0.81      0.75      1136
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.46      0.67      0.54       589
          12       0.55      0.76      0.64      1172
          13       0.63      0.62      0.63      1264
          14       0.29      0.12      0.17       343
          15       0.73      0.81      0.77       660
          16       0.58      0.84      0.69       885
          17       0.46    

Acc score:  0.22659531066260288


In [26]:
predict(tresh = 0.33)

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.68      0.76      0.72      1018
           4       0.82      0.82      0.82       806
           5       0.63      0.84      0.72       883
           6       0.68      0.82      0.74       944
           7       0.72      0.67      0.69       815
           8       0.73      0.76      0.75      1136
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.51      0.61      0.56       589
          12       0.60      0.70      0.64      1172
          13       0.68      0.55      0.61      1264
          14       0.38      0.09      0.14       343
          15       0.79      0.77      0.78       660
          16       0.62      0.82      0.71       885
          17       0.51    

Acc score:  0.25872951114737575


In [16]:
predict(tresh = 0.35)

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         0
           4       0.34      0.37      0.36        84
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          13       0.40      0.09      0.15        22
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         2
          17       0.19    

In [24]:
predict(tresh = 0.5)

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         6
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         0
           4       0.58      0.13      0.21        84
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         0
          13       1.00      0.09      0.17        22
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         2
          17       0.00    

In [42]:
df_train.columns[1:]

Index(['cs.LG', 'quant-ph', 'hep-ph', 'hep-th', 'cs.CV', 'gr-qc',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'astro-ph.CO', 'stat.ML',
       ...
       'cond-mat', 'alg-geom', 'q-alg', 'q-bio', 'dg-ga', 'adap-org',
       'chao-dyn', 'funct-an', 'solv-int', 'patt-sol'],
      dtype='object', length=166)

In [32]:
hard_abstract = 'Despite recent progress in generative adversarial network(GAN)-based vocoders, where the model generates raw waveform conditioned on mel spectrogram, it is still challenging to synthesize high-fidelity audio for numerous speakers across varied recording environments. In this work, we present BigVGAN, a universal vocoder that generalizes well under various unseen conditions in zero-shot setting. We introduce periodic nonlinearities and anti-aliased representation into the generator, which brings the desired inductive bias for waveform synthesis and significantly improves audio quality. Based on our improved generator and the state-of-the-art discriminators, we train our GAN vocoder at the largest scale up to 112M parameters, which is unprecedented in the literature. In particular, we identify and address the training instabilities specific to such scale, while maintaining high-fidelity output without over-regularization. Our BigVGAN achieves the state-of-the-art zero-shot performance for various out-of-distribution scenarios, including new speakers, novel languages, singing voices, music and instrumental audio in unseen (even noisy) recording environments. We will release our code and model at: this https URL'

In [44]:
abstracts = [
    hard_abstract
]

In [46]:
df_predict = pd.DataFrame({'abstract': abstracts})

for cat in df_train.columns[1:]:
    df_predict[cat] = len(abstracts) * [0]

/tmp/wsuser/ipykernel_382/3092842459.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_predict[cat] = len(abstracts) * [0]


In [47]:
df_predict

,abstract,cs.LG,quant-ph,hep-ph,hep-th,cs.CV,gr-qc,cond-mat.mes-hall,cond-mat.mtrl-sci,astro-ph.CO,...,cond-mat,alg-geom,q-alg,q-bio,dg-ga,adap-org,chao-dyn,funct-an,solv-int,patt-sol
0,Despite recent progress in generative adversar...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# import pytorch_lightning as pl
class PredictArxivAbstractsModule(pl.LightningDataModule):
    
    def __init__(self, df_predict, tokenizer, batch_size=1, max_token_len=256):
        super().__init__()
        self.df_predict = df_predict
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.predict_dataset = ArxivAbstracts(df=self.df_predict, tokenizer=self.tokenizer, max_len= self.max_token_len)

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, batch_size=self.batch_size)


In [87]:
   abstracts = [abstract]
    df_predict = pd.DataFrame({'abstract': abstracts}) 
    categories = df_train.columns[1:]
    for cat in categories:
        df_predict[cat] = len(abstracts) * [0]
        
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    arxiv_data = PredictArxivAbstractsModule(df_predict, tokenizer, max_token_len=384)
    arxiv_data.setup()

In [50]:
preds = trainer.predict(model, dataloaders=arxiv_data.predict_dataloader())

Predicting ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1 0:00:00 • 0:00:00 0.00it/s

In [66]:
def predict(abstract):
    abstracts = [abstract]
    df_predict = pd.DataFrame({'abstract': abstracts}) 
    categories = df_train.columns[1:]
    for cat in categories:
        df_predict[cat] = len(abstracts) * [0]
        
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    arxiv_data = PredictArxivAbstractsModule(df_predict, tokenizer, max_token_len=384)
    arxiv_data.setup()
    
    predictions = trainer.predict(model, dataloaders=arxiv_data.predict_dataloader())
    predictions = predictions[0].numpy()[0]
        
    predictions_proba = []
    predictions_labels = []
        
    for cat, prob in zip(categories, predictions):
        if prob >= 0.25:
            predictions_labels.append(cat)
            predictions_proba.append(prob)
        
    return list(zip(predictions_labels, predictions_proba))

In [67]:
predict(hard_abstract)

Predicting ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1 0:00:00 • 0:00:00 0.00it/s

[('cs.RO', 0.81435853),
 ('cs.DC', 0.9684285),
 ('physics.app-ph', 0.9479388),
 ('funct-an', 0.27130815)]

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [68]:
def predict_abstract(model, abstract):
    text = abstract
    
    encoding = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=384,
        padding="max_length",
        return_token_type_ids=False,
        return_attention_mask= True,
        truncation=True,
        return_tensors='pt'
      )
    
    test_prediction = model(ids = encoding["input_ids"], mask = encoding["attention_mask"])
    test_prediction = torch.sigmoid(test_prediction).flatten().detach().numpy()
    
    predictions_proba = []
    predictions_labels = []

    for cat, prob in zip(sorted(LABEL_COLUMNS), test_prediction):
        if prob >= 0.25:
            predictions_labels.append(cat)
            predictions_proba.append(prob)
    
    return sorted(list(zip(predictions_labels, predictions_proba)), key=lambda elem: elem[1])


In [69]:
abstract = 'Despite recent progress in generative adversarial network(GAN)-based vocoders, where the model generates raw waveform conditioned on mel spectrogram, it is still challenging to synthesize high-fidelity audio for numerous speakers across varied recording environments. In this work, we present BigVGAN, a universal vocoder that generalizes well under various unseen conditions in zero-shot setting. We introduce periodic nonlinearities and anti-aliased representation into the generator, which brings the desired inductive bias for waveform synthesis and significantly improves audio quality. Based on our improved generator and the state-of-the-art discriminators, we train our GAN vocoder at the largest scale up to 112M parameters, which is unprecedented in the literature. In particular, we identify and address the training instabilities specific to such scale, while maintaining high-fidelity output without over-regularization. Our BigVGAN achieves the state-of-the-art zero-shot performance for various out-of-distribution scenarios, including new speakers, novel languages, singing voices, music and instrumental audio in unseen (even noisy) recording environments. We will release our code and model at: this https URL'

predict_abstract(model, abstract)

[('stat.ML', 0.27130842),
 ('cs.LG', 0.8143588),
 ('eess.AS', 0.9479388),
 ('cs.SD', 0.9684285)]

In [73]:
abstract = "Function as a Service (FaaS) paradigm is getting widespread and is envisioned to be the next generation of cloud computing systems that mitigates the burden for programmers and cloud solution architects. However, FaaS does not explicitly deal with the data, and developers have to intervene and undergo the burden of managing the application data, often, via separate cloud services (e.g., AWS RDS). We overcome this problem of FaaS by borrowing the notion of object from the object-oriented programming into the serverless systems. We propose a new paradigm on top of the function abstraction, known as Object as a Service (OaaS), that offers encapsulation and abstraction benefits. The OaaS incorporates the application data into the object abstraction and relieves the developers from dealing with separate cloud services for the data management. It also unlocks opportunities for built-in optimization features, such as software reusability, data locality, and caching. Moreover, OaaS enables dataflow programming such that the developers define a workflow of functions transparently without getting involved into the synchronization and parallelism aspects. We implemented a prototype of the OaaS platform that is low-overhead and scalable. We evaluated it under real-world settings in terms of the ease-of-use, imposed overhead, and scalability. The results demonstrate that OaaS streamlines cloud programming and offers scalability with a minor overhead to the underlying cloud system"

In [74]:
# https://arxiv.org/abs/2206.05361

In [75]:
predict_abstract(model, abstract)

[('cs.SE', 0.5767369), ('cs.DC', 0.7603023)]